# LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import pymc3 as pm
import theano.tensor as tt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import yfinance as yf

WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable DF
WARN: Could not locate executable efl


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# INPUT TICKER

In [2]:
ticker = input('Type in the ticker of the stock: ')

Type in the ticker of the stock: AAPL


# INPUT START & END DATES TO DOWNLOAD DATA

In [3]:
start_date = input('Start date (YYYY-MM-DD): ')
end_date = input('End date (YYYY-MM-DD): ')

Start date (YYYY-MM-DD): 2021-01-01
End date (YYYY-MM-DD): 2022-12-31


# DOWNLOADING DATA

In [4]:
data = yf.download(ticker,start_date,end_date)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.503624,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,129.080048,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.735016,155088000
2021-01-07,128.360001,131.630005,127.860001,130.919998,128.991394,109578200
2021-01-08,132.429993,132.630005,130.229996,132.050003,130.104767,105158200
...,...,...,...,...,...,...
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.477127,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,129.652435,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,125.674026,85438400


# PREPROCESS

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Adj Close'].values.reshape(-1, 1))

# SPLIT DB

In [6]:
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

# BNN MODEL

In [7]:
def construct_bnn_model(X, y):
    n_input = X.shape[1]
    n_output = y.shape[1]
    
    with pm.Model() as bnn:
        # Priors for the neural network weights and biases
        init_w = pm.Normal('init_w', mu=0, sd=1, shape=(n_input, 20))
        init_b = pm.Normal('init_b', mu=0, sd=1, shape=(20,))
        
        weights_in = tt.nnet.relu(init_w)
        biases_in = tt.nnet.relu(init_b)
        
        init_w2 = pm.Normal('init_w2', mu=0, sd=1, shape=(20, n_output))
        init_b2 = pm.Normal('init_b2', mu=0, sd=1, shape=(n_output,))
        
        weights_out = tt.nnet.relu(init_w2)
        biases_out = tt.nnet.relu(init_b2)
        
        # Neural network with ReLU activation
        act_1 = tt.nnet.relu(tt.dot(X, weights_in) + biases_in)
        act_out = tt.dot(act_1, weights_out) + biases_out
        
        # Likelihood of the observed data
        obs = pm.Normal('obs', mu=act_out, sd=0.1, observed=y)
        
    return bnn

# PREPARE TRAINING

In [8]:
X_train, y_train = [], []
n_future = int(input('Number of days to predict: '))

for i in range(len(train_data)-n_future):
    X_train.append(train_data[i:i+n_future])
    y_train.append(train_data[i+n_future])

X_train, y_train = np.array(X_train), np.array(y_train)

Number of days to predict: 5


# FLATTEN THE DATA

In [9]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]))

# RUNNING MODEL

In [29]:
bnn_model = construct_bnn_model(X_train, y_train)

# BAYESIAN INFERENCE

In [30]:
with bnn_model:
    trace = pm.sample(draws=500, tune=500)

C:\Users\bviei\anaconda3\lib\site-packages\deprecat\classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [init_b2, init_w2, init_b, init_w]


Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 2339 seconds.
There were 382 divergences after tuning. Increase `target_accept` or reparameterize.
There were 343 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


# PREPARE TESTING

In [31]:
X_test, y_test = [], []

for i in range(len(test_data) - n_future):
    X_test.append(test_data[i:i+n_future])
    y_test.append(test_data[i+n_future])

X_test, y_test = np.array(X_test), np.array(y_test)

# FLATTEN TEST DATA

In [32]:
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1]))

# MAKE PREDICTIONS

In [33]:
with bnn_model:
    posterior_pred = pm.sample_posterior_predictive(trace, samples=200)

C:\Users\bviei\anaconda3\lib\site-packages\pymc3\sampling.py:1708: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


# MEAN PREDICTIONS

In [34]:
mean_predictions = posterior_pred['obs'].mean(axis=0)
print(mean_predictions)

[[0.21747114]
 [0.1860931 ]
 [0.19272597]
 [0.21825224]
 [0.19080405]
 [0.17076028]
 [0.1775265 ]
 [0.2352427 ]
 [0.30968713]
 [0.32416311]
 [0.40678429]
 [0.39751421]
 [0.37671043]
 [0.33189715]
 [0.24109748]
 [0.26230611]
 [0.27744851]
 [0.26302375]
 [0.31462204]
 [0.29834348]
 [0.30249271]
 [0.29598935]
 [0.28724077]
 [0.29345897]
 [0.30548977]
 [0.26544101]
 [0.2338109 ]
 [0.21056724]
 [0.2057977 ]
 [0.15789811]
 [0.14622888]
 [0.1421257 ]
 [0.06614503]
 [0.07317289]
 [0.16875437]
 [0.13245014]
 [0.09067671]
 [0.07020527]
 [0.07948527]
 [0.00447084]
 [0.06997573]
 [0.06151777]
 [0.07215116]
 [0.05472712]
 [0.11094705]
 [0.13948807]
 [0.13617267]
 [0.07326708]
 [0.04163469]
 [0.10744891]
 [0.09032068]
 [0.05387225]
 [0.05759905]
 [0.07356243]
 [0.09372181]
 [0.05439497]
 [0.0821123 ]
 [0.08985285]
 [0.13334761]
 [0.14886892]
 [0.16017693]
 [0.20858097]
 [0.25247489]
 [0.21397637]
 [0.26402065]
 [0.24041126]
 [0.25863981]
 [0.26056403]
 [0.26196904]
 [0.23879375]
 [0.25879271]
 [0.25

# INVERSE TRANSFORM

In [35]:
mean_predictions = scaler.inverse_transform(mean_predictions.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
print(y_test_inv)

[[172.24186707]
 [173.75492859]
 [173.35673523]
 [170.73872375]
 [166.80671692]
 [166.46826172]
 [166.76690674]
 [169.25550842]
 [162.87472534]
 [160.64491272]
 [158.18617249]
 [156.50386047]
 [157.24049377]
 [155.10028076]
 [153.8261261 ]
 [155.24961853]
 [153.75643921]
 [156.65318298]
 [162.68557739]
 [153.13925171]
 [154.60255432]
 [151.67596436]
 [150.01356506]
 [153.77633667]
 [156.18531799]
 [153.01980591]
 [152.04428101]
 [149.74479675]
 [150.08325195]
 [151.06874084]
 [149.1574707 ]
 [141.83099365]
 [137.57048035]
 [141.80114746]
 [145.43452454]
 [145.73313904]
 [144.76756287]
 [139.45188904]
 [139.78039551]
 [138.34693909]
 [137.70985413]
 [142.33868408]
 [137.74967957]
 [141.76133728]
 [143.0952301 ]
 [143.20472717]
 [142.73686218]
 [146.59918213]
 [148.76924133]
 [151.64608765]
 [148.66970825]
 [144.14044189]
 [155.03062439]
 [152.64154053]
 [149.96379089]
 [144.36938477]
 [138.24740601]
 [137.97817993]
 [138.51661682]
 [139.09494019]
 [134.4783783 ]
 [146.44352722]
 [149.26

# RMSE

In [36]:
y_test_inv = y_test_inv[:len(mean_predictions)]
rmse = np.sqrt(mean_squared_error(y_test_inv, mean_predictions))
print('RMSE: ', rmse)

ValueError: Found input variables with inconsistent numbers of samples: [96, 397]

In [ ]:


# Calculate the root mean squared error
rmse = np.sqrt(mean_squared_error(y_test_inv, mean_predictions))
print('RMSE:', rmse)